In [1]:
from flu_model import FluModel, State
import numpy as np
import pandas as pd
from selenium.webdriver import Chrome
from ipywidgets.embed import embed_minimal_html

#web_driver = Chrome('/path/to/chromedriver')
#web_driver.set_page_load_timeout(200)

# ipywidgets for UI
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Label

# Bokeh (visual) imports
from bokeh.io import output_notebook, show, export_svgs, push_notebook
from bokeh.models import ColumnDataSource, Legend, LegendItem, Range1d
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.layouts import row
from bokeh.models.annotations import Title

# silence missing renderers warning (empty plot must be set for updating)
from bokeh.core.validation.warnings import MISSING_RENDERERS
from bokeh.core.validation import silence
silence(MISSING_RENDERERS, True)

# set jupyter output
output_notebook()

# set color scheme
colors = ['#081480', '#16ca02', '#ffce25', '#e011f7', '#16ca02']

Loading BokehJS ...

In [2]:
def plot_bokeh(p, model, N_steps, deceased, sampled, plots):

    # get agent model results
    agent_state = model.datacollector.get_agent_vars_dataframe().drop(columns='p')
    
    # Change into more readable categorical data
    df = pd.pivot_table(agent_state.reset_index(),index='Step',columns='State',aggfunc=np.size, fill_value=0)
    labels = ['Susceptible','Infected','Recovered']
    
    df.columns = labels[:len(df.columns)]
    df.reset_index(inplace=True);

    # add columns to DataFrame if neccessary 
    if plots[2] and 'Recovered' not in df.columns:
        df = pd.concat([df, pd.Series([0] * len(df.index), name='Recovered')], axis=1)
    if plots[3]:
        df = pd.concat([df, pd.Series(deceased, name='Deceased')], axis=1)
    if plots[4]:
        df = pd.concat([df, pd.Series(sampled, name='Sampled')], axis=1)
    
    # create ColumnDataSource object for Bokeh formatting
    source = ColumnDataSource(df)
        
    # Add each line to the figure and append to legend_items
    for i, c in enumerate(['Susceptible', 'Infected', 'Recovered', 'Deceased', 'Sampled']):
        
        # for columns checked in UI
        if plots[i]:
            
            if c == 'Sampled':
                p.line(x='Step', y=c, source=source, line_color=colors[1], 
                               line_dash='dotted', line_width=1.5, line_alpha=0.85,
                               legend_label='Positive Cases')

            else:
                p.line(x='Step', y=c, source=source,
                               line_color=colors[i], line_width=3,
                               line_alpha=0.85, legend_label=c)
    
    # set up legend on first run only
    if len(df.index) == 1:
        p.legend.border_line_color = 'black'
        p.legend.border_line_width = 2
        p.legend.location = 'top_right'
        p.legend.background_fill_alpha = 0.35
        p.legend.label_text_font_size = '9pt'
        p.legend.spacing = 0
        
    return p

In [3]:
def plot_grid(p, model, color_map):
    
    p.renderers = []
    
    x_vals = []; y_vals = []; states = []
    
    # get occupants of each cell in the grid
    for cell in model.grid.coord_iter():
        agents, x, y = cell
        
        for agent in agents:
            x_vals.append(int(x))
            y_vals.append(int(y))
            states.append(str(agent.state))
            
    df = pd.DataFrame({'x': x_vals, 'y': y_vals, 'state': states})
    df = df.dropna()
    
    source2 = ColumnDataSource(data=df)

    # fill in rectangles at (x,y) location w/ color mapped from present state
    p.rect(x='x', y='y', width=1, height=1, source=source2,
        fill_color=color_map, fill_alpha=0.75, line_color='black')
    
    return p

In [4]:
def make_p1(opts, pop, N_steps, plots):
    
    p1 = figure(**opts)
    p1.output_backend = 'svg'

    p1.y_range=Range1d(0, pop)
    p1.x_range=Range1d(0, N_steps)
    
    p1.background_fill_color = "#cdcdcd"
    p1.background_fill_alpha = 0.3
    
    p1.toolbar.logo = None
    p1.toolbar_location = None
    
    p1.xaxis.axis_label = 'Time'
    p1.yaxis.axis_label = 'People'
    
    p1.xaxis.major_label_text_color = None
    p1.yaxis.major_label_text_color = None 
    
    t1 = Title(); t1.text = 'SIR Model of Pandemic Response'
    p1.title = t1
    p1.title.text_font_size = "24px"
    
    return p1

In [5]:
def make_p2(opts, x_map, y_map):
    
    p2 = figure(**opts)
    p2.output_backend = 'svg'
    
    p2.x_range=Range1d(-1,x_map)
    p2.y_range=Range1d(-1,y_map)
    
    p2.background_fill_color = '#D0D0D0'
    p2.background_fill_alpha = 0.76

    p2.toolbar.logo = None
    p2.toolbar_location = None
    
    p2.xaxis.axis_label = None
    p2.yaxis.axis_label = None

    p2.xaxis.major_tick_line_color = None
    p2.xaxis.minor_tick_line_color = None
    
    p2.yaxis.major_tick_line_color = None
    p2.yaxis.minor_tick_line_color = None 
    
    p2.axis.axis_line_color = None
    p2.grid.grid_line_color = None
    
    p2.xaxis.major_label_text_color = None
    p2.yaxis.major_label_text_color = None 
        
    return p2

In [6]:
def flu_simulation(pop, x_map, y_map, s, opts, N_steps, biased, init_inf, plots=[True, True, True, True, True]):
    
    # generate model 
    model = FluModel(pop, width=x_map, height=y_map, init_inf=init_inf, biased=biased)
    
    # initialize color map and empty lists
    factors = ['State.SUSCEPTIBLE', 'State.INFECTED', 'State.RECOVERED']
    color_map = factor_cmap(field_name='state', palette=colors, factors=factors)
    deceased = []; sampled = []
    
    # create time + grid plots
    p1 = make_p1(opts, pop, N_steps, plots)
    p2 = make_p2(opts, x_map, y_map)
        
    # show both plots and save handle for updating
    t = show(row(p1,p2), notebook_handle=True)

    for i in range(N_steps):
        
        # advance the flu model
        model.step()

        # append number of deceased from model to list
        deceased.append(model.deceased)

        # sample data for plotting 
        sampled.append(model.sample(s,i) * 100/s)

        # get repainted plots
        p1 = plot_bokeh(p1, model, N_steps, deceased, sampled, plots)
        p2 = plot_grid(p2, model, color_map)
        
        # export svgs
        #export_svgs(p1, filename="imgs/test_plot" + str(i) + ".svg", webdriver=web_driver, timeout=100)
        #export_svgs(p2, filename='imgs/test_grid' + str(i) + '.svg', webdriver=web_driver, timeout=100)

        # push changes to jupyter notebook
        push_notebook(handle=t)

    # OUTPUT .CSV OF RESULTS
    # get agent model results
    agent_state = model.datacollector.get_agent_vars_dataframe().drop(columns='p')

    # change into counts of categorical data, rename columns
    df = pd.pivot_table(agent_state.reset_index(),index='Step',columns='State',aggfunc=np.size, fill_value=0)
    df.columns = ['Susceptible', 'Infected', 'Recovered']
    
    # add deceased and sampled (dashed-line) data to the dataframe
    df['Deceased'] = deceased
    df['Sampled'] = sampled
            
    df.to_csv('output.csv')
        

In [7]:
## USER INTERFACE

# create buttons 
button = widgets.Button(description='Run')
button2 = widgets.Button(description='Fit to Population')

# gather selected values upon button push // submission
def launch_flu_simulation(b=None):
            
    # get checkbox values + assemble for plotting
    su_cb = susceptible.value
    in_cb = infected.value
    re_cb = recovered.value
    de_cb = deceased.value
    sa_cb = sampled.value
    
    plots = [su_cb, in_cb, re_cb, de_cb, sa_cb]        
        
    # build opts for plotting
    opts = dict(plot_width=490, plot_height=490, min_border=0, toolbar_location=None)
    
    # launch mesa simulation
    flu_simulation(pop=population_box.value, x_map=map_size_x.value, y_map=map_size_y.value,
                       s=sample_slider.value, opts=opts, N_steps=steps_box.value, 
                       init_inf=infected_slider.value, plots=plots, biased=biased_check.value)

# button click actions
@button.on_click
def plot_on_click(b):
    launch_flu_simulation()
    
@button2.on_click
def calc_on_click(b):
    
    # get population and find appropriate square dimensions
    pop = population_box.value
    dim = int(np.ceil(np.sqrt(pop) * 1.4))
    
    # update fields
    map_size_x.value = dim
    map_size_y.value = dim

# set up first tab input fields
sample_slider = widgets.FloatSlider(
    value=1,
    min=0,
    max=100.0,
    step=0.1,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

biased_check = widgets.Checkbox()

# first tab for sampling info
tab1 = VBox(children=[HBox([Label('Percent Sampled'), sample_slider]),
                      HBox([Label('Biased Sampling'), biased_check])])

# set up second tab input fields
population_box = widgets.BoundedIntText(
    value=300,
    min=10,
    max=10000,
    step=1,
    orientation='horizontal'
)

steps_box = widgets.BoundedIntText(
    value=90,
    min=10,
    max=300,
    step=1,
    orientation='horizontal'
)

infected_slider = widgets.FloatSlider(min=0.1, max=5, step=0.1, value=1)

map_size_x = widgets.BoundedIntText(
    value=25,
    max=1000,
    min=10,
    step=1,
    orientation='horizontal'
)

map_size_y = widgets.BoundedIntText(
    value=25,
    min=10,
    max=1000,
    step=1,
    orientation='horizontal'
)

# second tab for model details
tab2 = VBox(children=[HBox([Label('Population'), population_box]), HBox([Label('Number of Timesteps'), steps_box]),
                              HBox([Label('Initial Infected Percent'), infected_slider]),
                              HBox([Label('Map Size'), map_size_x, Label(' x '), map_size_y, button2])])

# set up third tab input fields
susceptible = widgets.Checkbox(value=True)
infected = widgets.Checkbox(value=True)
recovered = widgets.Checkbox(value=True)
deceased = widgets.Checkbox(value=True)
sampled = widgets.Checkbox(value=True)

output_fn = widgets.Text(
    placeholder='Enter filename',
    disabled=False
)

# third tab for plot displays
tab3 = HBox([VBox([Label('Susceptible'), Label('Infected'), Label('Recovered'), Label('Deceased'), Label('Sampled')]),
             VBox([susceptible, infected, recovered, deceased, sampled])])

# create complete user interface
tab = widgets.Tab(children=[tab1, tab2, tab3])
tab.set_title(0, 'Sampling')
tab.set_title(1, 'Model')
tab.set_title(2, 'Plotting')
VBox(children=[tab, button])